In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.ProteinSignallingTransductionWithoutKm() 

In [ ]:
spl_t_0, spl_t_T, spl_freq = 0, 100, 20
obs_t_0, obs_t_T, obs_freq = 0, 100, 20
est_t_0, est_t_T, est_freq = 0, 100, 20
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
t_indices = np.array([0, 1, 2, 4, 5, 7, 10, 15, 20, 30, 40, 50, 60, 80, 100]) * spl_freq
obs_tps = obs_tps[t_indices]
obs_t_indices = obs_t_indices[t_indices]
est_tps = est_tps[t_indices]
est_t_indices = est_t_indices[t_indices]
X_0 = np.array([1., 0., 1., 0., 0.]) 
theta = np.array([0.07, 0.6, 0.05, 0.3, 0.017]) 
rho_2 = None
sigma_2 = np.full(dynamical.num_x, 1e-2) 
delta = np.full(dynamical.num_x, True) 
# gamma = np.full(dynamical.num_x, 5e-3) 
gamma = np.array([1e-4, 1e-4, 2e-6, 1e-5, 5e-2])
phi = [
    # (Kernal name, Kernal parameters)
    ('sigmoid', np.array([1., .4, 15.])),
    ('sigmoid', np.array([.18, .6, 25.])),
    ('sigmoid', np.array([.84, 3., 3.1])),
    ('sigmoid', np.array([.62, 2., 2.1])),
    ('sigmoid', np.array([.84, 3., 2.9])),
]

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = True
plotting_freq = 50
plotting_config = {
    'x': {
        'xlim': (0, 100),
        'ylim': (0., 1.2)
    }
}

In [ ]:
num_repetitions = 10
directory = '../data/ode-protein-without-km/'
config_filename = utils.CONFIG_FILENAME
data_filename = utils.DATA_FILENAME

In [ ]:
spl_X = dynamical.generate_sample_path(theta, rho_2, X_0, spl_tps)

In [ ]:
utils.save_ode_config(directory, config_filename,
                      spl_t_0, spl_t_T, spl_freq, spl_tps,
                      obs_t_0, obs_t_T, obs_freq, obs_tps, obs_t_indices,
                      est_t_0, est_t_T, est_freq, est_tps, est_t_indices,
                      X_0, theta, rho_2, phi, sigma_2, delta, gamma,
                      opt_method, opt_tol, max_init_iter, max_iter,
                      plotting_enabled, plotting_freq, spl_X)

In [ ]:
(spl_t_0, spl_t_T, spl_tps, spl_freq,
 obs_t_0, obs_t_T, est_freq, obs_tps, obs_t_indices,
 est_t_0, est_t_T, est_freq, est_tps, est_t_indices,
 X_0, theta, rho_2, phi, sigma_2, delta, gamma,
 opt_method, opt_tol, max_init_iter, max_iter, plotting_enabled, plotting_freq,
 spl_X) = utils.load_ode_config(directory, config_filename)

In [ ]:
for repetition in range(1, num_repetitions + 1):
    print('Starting repetition {}'.format(i))    
    obs_Y = utils.collect_observations(spl_X, obs_t_indices, sigma_2, [0, None])
    data = core.laplace_mean_field(dynamical, 
                                   spl_X, spl_tps, 
                                   obs_Y, obs_tps, obs_t_indices, 
                                   est_tps, est_t_indices,
                                   theta, rho_2, phi, sigma_2, delta, gamma,
                                   opt_method, opt_tol, max_init_iter, max_iter,
                                   plotting_enabled, plotting_freq)
    data['obs_Y'] = obs_Y
    utils.save_data(directory, data_filename.format(repetition), data)

In [ ]:
data = []
for i in range(num_repetitions):
    data.append(utils.load_data(directory, data_filename.format(i + 1)))

In [ ]:
figure = plt.figure(figsize=plt.figaspect(5 / 9))

bar_width = 0.3
bar_indices = np.arange(theta.size)
ax = figure.add_subplot(2, 3, 1)
ax.bar(bar_indices, theta, bar_width, color='C0', label='Truth')
ax.bar(bar_indices + bar_width, utils.get_theta_mean(data), bar_width, color='C2', label='Estimation',
               yerr= 2 * np.sqrt(utils.get_theta_var(data)),
               error_kw=dict(ecolor='0.5', capsize=3., capthick=.5))
ax.set_ylabel('Value')
ax.set_title('Theta')
ax.set_xticks(bar_indices + bar_width / 2)
ax.set_xticklabels([r'${}$'.format(label) for label in dynamical.theta_labels])
ax.set_ylim([0, .8])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc=0)

X_mean = utils.get_X_mean(data)
X_var = utils.get_X_var(data)

for i in range(dynamical.num_x):
    ax = figure.add_subplot(2, 3, 2 + i)
    ax.plot(spl_tps, spl_X[i], color='C0', linestyle='-', linewidth=1.5, label='Sample path')
    if delta[i]:
        ax.scatter(obs_tps, obs_Y[i], color='C1', marker='x', label='Observation')
    ax.errorbar(est_tps, X_mean[i], color='C2', linestyle='--', linewidth=1.5, label='Estimation', 
                yerr=2 * np.sqrt(X_var[i]), ecolor='0.5', capsize=3., capthick=.5)
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.set_title(r'State ${}$'.format(dynamical.x_labels[i]))
    ax.set_xlim([spl_t_0, spl_t_T])
    ax.set_ylim([0, 1.1])
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles, labels=labels, loc=0)

figure.tight_layout()

plt.show()